https://github.com/google-research/bert

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import re
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/pretrained-bert-including-scripts/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/vocab.txt
/kaggle/input/pretrained-bert-including-scripts/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
/kaggle/input/pretrained-bert-including-scripts/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.index
/kaggle/input/pretrained-bert-including-scripts/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.meta
/kaggle/input/pretrained-bert-including-scripts/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_config.json
/kaggle/input/pretrained-bert-including-scripts/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16/vocab.txt
/kaggle/input/pretrained-bert-including-scripts/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001
/kaggle/input/pretrained-bert-including-scripts/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16/bert_model.ckpt.index
/kaggle/input/pretrained-bert-including-scripts/unca

In [2]:
# ! pip install -U tokenizers

In [3]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 28 kB/s  eta 0:00:011|████▋                           | 59.9 MB 50.3 MB/s eta 0:00:08     |█████████████████████████████   | 373.9 MB 47.2 MB/s eta 0:00:01     |█████████████████████████████   | 374.2 MB 47.2 MB/s eta 0:00:01     |██████████████████████████████▏ | 388.7 MB 47.2 MB/s eta 0:00:01     |██████████████████████████████▎ | 389.8 MB 7.7 MB/s eta 0:00:03     |██████████████████████████████▍ | 392.1 MB 7.7 MB/s eta 0:00:03     |██████████████████████████████▌ | 393.2 MB 7.7 MB/s eta 0:00:03     |███████████████████████████████▋| 407.7 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 39.7 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorboa

In [4]:
Path = '/kaggle/input/hebrew-corpus-oscar/he_dedup.txt'

# **Cleaning text**

In [5]:
# #remove punctutaion
# ' '.join(word.strip(string.punctuation) for word in rrr.split())
# #remove chinese characters in tokenizers
# #remove non hebrew character
# [w for w in rrr if not re.match(r'[A-Za-z]+', w, re.I)]
# #remove numbers




# Sample - 1000 rows

In [6]:
with open(Path) as myfile:
    head = [next(myfile) for x in range(1000)]
# print(head)

**Write sample to file txt**

In [7]:
outF = open("mysample.txt", "w")
for line in head:
  # write line to output file
  outF.write(line)
  outF.write("\n")
outF.close()

In [8]:
import tokenizers

In [9]:
bwpt = tokenizers.BertWordPieceTokenizer(
    vocab_file=None,
    add_special_tokens=True,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)

Split original file to mini files

In [10]:
# !split -l 15000 /kaggle/input/hebrew-corpus-oscar/he_dedup.txt hebrew_
# import glob
# file_list = glob.glob("..../hebrew_*")

# Train the tokenizer

In [11]:
bwpt.train(
#     files=["../input/hindi-oscar-corpus/hi_dedup_1000.txt"],
    files = ["../working/mysample.txt"],
    vocab_size=30000,
    min_frequency=3,
    limit_alphabet=1000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

**Save vocab**

In [12]:
bwpt.save("../working/", "hebrew")

['../working/hebrew-vocab.txt']

let's check it out

In [13]:
encoded = bwpt.encode("היי, מה המצב?")
print(encoded)

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing, original_str, normalized_str])


In [14]:
cd ../input/bertsrc/

/kaggle/input/bertsrc


In [15]:
!ls

CONTRIBUTING.md		    optimization_test.py
LICENSE			    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
README.md		    requirements.txt
__init__.py		    run_classifier.py
create_pretraining_data.py  run_classifier_with_tfhub.py
extract_features.py	    run_pretraining.py
modeling.py		    run_squad.py
modeling_test.py	    sample_text.txt
multilingual.md		    tokenization.py
optimization.py		    tokenization_test.py


# Create pretraining data

In [16]:
!python create_pretraining_data.py \
    --input_file=/kaggle/working/mysample.txt \
    --output_file=/kaggle/working/tf_examples.tfrecord \
    --vocab_file=/kaggle/working/hebrew-vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5

W0421 11:32:47.797061 140538879149440 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0421 11:32:47.797339 140538879149440 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0421 11:32:47.797619 140538879149440 module_wrapper.py:139] From /kaggle/input/bertsrc/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0421 11:32:47.839276 140538879149440 module_wrapper.py:139] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0421 11:32:47.841992 140538879149440 module_wrapper.py:139] From create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0421 11:32:47.842214 140538879149440 create_pretraining_data.py:4

I0421 11:32:54.967774 140538879149440 create_pretraining_data.py:161] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
I0421 11:32:54.967960 140538879149440 create_pretraining_data.py:161] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
I0421 11:32:54.968159 140538879149440 create_pretraining_data.py:161] masked_lm_positions: 22 25 26 30 35 36 46 54 60 69 76 80 87 96 100 102 108 121 125 0
I0421 11:32:54.968347 140538879149440 create_pretraining_data.py:161] masked_lm_ids: 1635 744 3876 2036 788 743 2426 16 3037 328 327 285 237 348 257 302 228 307 341 0
I

I0421 11:32:56.411960 140538879149440 create_pretraining_data.py:166] Wrote 5000 total instances


# Run pretraining data

In [17]:
BERT_PARAM='/kaggle/input/pretrained-bert-including-scripts/cased_L-12_H-768_A-12/cased_L-12_H-768_A-12'

In [19]:
!python run_pretraining.py \
  --input_file=/kaggle/working/tf_examples.tfrecord \
  --output_dir=/kaggle/working/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=$BERT_PARAM/bert_config.json \
  --init_checkpoint=$BERT_PARAM/bert_model.ckpt \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=30 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5

W0421 11:40:47.475868 140390511695232 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0421 11:40:47.476208 140390511695232 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0421 11:40:47.476577 140390511695232 module_wrapper.py:139] From /kaggle/input/bertsrc/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0421 11:40:47.478442 140390511695232 module_wrapper.py:139] From run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0421 11:40:47.478711 140390511695232 module_wrapper.py:139] From run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0421 11:40:47.480068 140390511695232 module_wrapper.py:139] From run_pretraining.py:420: The name tf.logg

W0421 11:40:50.223305 140390511695232 deprecation.py:506] From /kaggle/input/bertsrc/modeling.py:358: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0421 11:40:50.239844 140390511695232 deprecation.py:323] From /kaggle/input/bertsrc/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
W0421 11:40:50.241720 140390511695232 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W0421 11:40:52.376452 140390511695232 module_wrapper.py:139] From run_

I0421 11:40:53.202926 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_10/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:40:53.203093 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_10/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:40:53.203244 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_11/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0421 11:40:53.203478 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_11/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:40:53.203684 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_11/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0421 11:40:53.203891 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_11/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:40:53.204042 140390511695232 run_pretraining.py:173]   name 

W0421 11:41:06.865769 140390511695232 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/training/saver.py:1069: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.
I0421 11:41:07.867954 140390511695232 session_manager.py:500] Running local_init_op.
I0421 11:41:08.139971 140390511695232 session_manager.py:502] Done running local_init_op.
I0421 11:41:17.430602 140390511695232 basic_session_run_hooks.py:606] Saving checkpoints for 0 into /kaggle/working/pretraining_output/model.ckpt.
I0421 11:42:39.596090 140390511695232 tpu_estimator.py:2307] global_step/sec: 0.0262071
I0421 11:42:39.597620 140390511695232 tpu_estimator.py:2308] examples/sec: 0.838628
I0421 11:43:11.979286 140390511695232 tpu_estimator.py:2307] global_step/sec: 0.0308802
I0421 11:43:11.980077 140390511695232 tpu_estimator.py:2308] examp

I0421 11:58:05.626397 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_5/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0421 11:58:05.626586 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0421 11:58:05.626762 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_5/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:58:05.626969 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_5/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:58:05.627161 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_5/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0421 11:58:05.627375 140390511695232 run_pretraining.py:173]   name = bert/encoder/layer_6/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0421 11:58:05.627560 140390511695232 run_pretraining.py:173]   name = bert/encoder/lay

I0421 11:58:05.720471 140390511695232 estimator.py:1150] Done calling model_fn.
I0421 11:58:05.743078 140390511695232 evaluation.py:255] Starting evaluation at 2020-04-21T11:58:05Z
I0421 11:58:06.339923 140390511695232 monitored_session.py:240] Graph was finalized.
2020-04-21 11:58:06.341219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-21 11:58:06.341999: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-04-21 11:58:06.342377: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2020-04-21 11:58:06.342568:

global_step = 30  # we need to run it for 10000 epochs
loss = 8.56319
masked_lm_accuracy = 0.021613833
masked_lm_loss = 8.562686
next_sentence_accuracy = 1.0
next_sentence_loss = 0.0010693793